![MIoT_GDPI](img/MIOT_GDPI_header.png)

# Unidad 02 - Operaciones comunes en Stream Learning

El objetivo principal de esta práctica es que os familiaricéis con las operaciones básicas para trabajar en *online learning* y como integrarlas en un *pipeline* de [River](https://riverml.xyz/). Podréis comprobar que su uso es similar al de `scikit-learn` que ya conocéis.

La mayor parte del contenido de este Notebook se dedica a explicar el uso de los comandos y de su integración en un *pipeline* apoyándose en ejemplos concretos que ilustran su aplicación a un problema real. Es crucial que dediquéis tiempo a leer y comprender el material, en lugar de simplemente ejecutar el código. Os invitamos a experimentar modificando y variando el código proporcionado para que podáis explorar las distintas opciones y profundizar en su funcionamiento.



**importante**: El Notebook contiene varios ejercicios sencillos. Debéis desarrollarlos durante la clase y enviarlos por el aula virtual del curso, en la tarea correspondiente.



## Referencias útiles para la práctica
1. API Pandas: [https://pandas.pydata.org/docs/reference/index.html](https://pandas.pydata.org/docs/reference/index.html)
2. Api River: [https://riverml.xyz](https://riverml.xyz)
3. Api Scikit-Learn: [https://scikit-learn.org/stable/api/index.html](https://scikit-learn.org/stable/api/index.html)
4. Bahri, M., Bifet, A., Gama, J., Gomes, H. M., & Maniu, S. (2021). [Data stream analysis: Foundations, major tasks and tools](https://doi.org/10.1002/widm.1405). Wiley Interdisciplinary Reviews: Data Mining and Knowledge Discovery, 11(3), e1405.
5. Gomes, H. M., Read, J., Bifet, A., Barddal, J. P., & Gama, J. (2019). [Machine learning for streaming data: state of the art, challenges, and opportunities](https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://kdd.org/exploration_files/3._CR_7._Machine_learning_for_streaming_data_state_of_the_art-Final.pdf). ACM SIGKDD Explorations Newsletter, 21(2), 6-22.


## 1. Avanzando de *batch ML* tradicional al *online ML*


Este tutorial está inspirado en los ejemplos proporcionados por [River](https://riverml.xyz/)en su página [web](https://riverml.xyz/latest/recipes/reading-data/)  Por lo tanto, os animamos  a consultar la documentación original para una exploración más profunda de cada uno de los temas particulares que se muestren a continuación.

En general, casi cualquier enfoque asociado al aprendizaje automático contiene los siguientes pasos:
1. Entender y definir el problema y su contexto.
2. Obtener los datos.
3. Explorar, analizar y entender los datos.
4. Preprocesar los datos
5. Seleccionar, optimizar y entrenar los modelos ML.
6. Evaluar y el modelo seleccionado.
7. Desplegar, monitorizar y mantener la solución.

Antes de comenzar, vamos a examinar un ejemplo clásico que utiliza todos estos pasos utilizando la librería  `scikit-learn`:

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, roc_auc_score
from sklearn.model_selection import cross_val_score, KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from rich import print


# Carga de datos
dataset = load_breast_cancer()
X, y = dataset.data, dataset.target

# Preparación del pipeline con los pasos de preprocesado y el modelo seleccionado
pipe = Pipeline([
    ('scale', StandardScaler()),                       # Estandarización de los datos avg 0 y std 1
    ('extractor', PCA(0.95)),                          # Reducción de la dimensionalidad con PCA (0.95 of the varianza)
    ('classifier', LogisticRegression(solver='lbfgs')) # Selección y configuración del modelo
])

# Definición de una validación cruzada estableciendo una semilla
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Evaluación del modelo para cada fold de la validación cruzada
scorer = make_scorer(roc_auc_score)
scores = cross_val_score(pipe, X, y, scoring=scorer, cv=cv)

# Imprime la media de la métrica y su desciación estándard
print(f'ROC AUC: {scores.mean():.4f} (± {scores.std():.4f})')

¿Cuál es el problema con esta aproximación? En realidad, no tiene ningún problema, pero existen algunas desventajas potenciales, particularmente con respecto al tamaño del conjunto de los datos, la aparición de datos nuevos y la heterogeneidad de las características a lo largo del tiempo.

* **Tamaño del conjunto de datos**: imagina que los datos para el entrenamiento fueran demasiado grandes para caber en la memoria de tu ordenador. En ese caso, el programa, probablemente,  se habría bloqueado. Aunque se pueden aplicar diferentes técnicas para minimizar este problema,  existe un límite para estas optimizaciones. De hecho, si el dataset constara de millones de observaciones (ej.  cientos de gigabytes), podría haberse requerido hardware especial solo para cargar el dataset.

* **Datos nuevos**: Otro problema potencial es la incorporación de  datos  nuevos al modelo. Los enfoques tradicionales requieren comenzar a entrenar desde cero con un nuevo conjunto de datos resultante de la combinación de los datos antiguos con las nuevas observaciones disponibles. Esto es particularmente problemático en aplicaciones en tiempo real donde tienes  datos nuevos disponibles cada poco. En muchas aplicaciones reales, la solución es desarrollar un pipeline de integración continua que pueda generar y desplegar un nuevo modelo cada poco. es importante destacar que por datos nuevos nos referimos a observaciones que son radicalmente diferentes de las empleadas para entrenar, dónde las relaciones entre las entradas (*inputs*) y salidas (*outputs*) ha cambiado de forma que el modelo no puede gestionarlas.

* **Heterogeneidad de características**: Finalmente, otra desventaja con el enfoque tradicional es que las características con las que se ha entrenado deberán estar disponibles siempre durante el tiempo de producción. Además, no pueden incorporarse nuevas características a lo largo del tiempo. En escenarios de plantas industriales es habitual que determinadas características no estén disponibles en un momento determinado (sensores en estropeados o con mal funcionamiento) para luego estar nuevamente disponibles. En estos mismos escenarios, nuevos sensores pueden desplegarse y empezar a medir y generar nuevas características. Desde un punto de vista tradicional, la única forma de gestionar un número cambiante de características es reentrenando desde cero.


## 2. *Incremental Learning*

Como se mencionó en la unidad anterior, el aprendizaje incremental (*incremental learning*) también se conoce como aprendizaje en línea (*online learning*) o aprendizaje de flujo (*stream learning*). El término *online* se sustituye habitualmente por incremental o *stream* debido a su confusión con el concepto de "aprendizaje en línea", que, generalmente,  se refiere a una opción educativa (esto es obvio si buscas en Google "aprendizaje en línea"). 

El concepto de aprendizaje incremental se refiere a ajustar un modelo de aprendizaje automático respecto a los datos obtenidos de flujo (*stream*) de datos. Expresado de otra manera, los datos no están disponibles en su totalidad, sino que las observaciones se proporcionan una por una a medida que se leen del flujo de de datos. Por ejemplo, imagina el ejemplo clásico anterior que, en lugar del conjunto de datos completo para entrenar, tenemos un punto de referencia temporal que proporciona una observación a la vez. Esto se puede simular con un simple bucle: 


In [ ]:
for xi, yi in zip(X, y):#simula obtener una observación por cada vuelta del bucle
    pass

print(xi)#Imprimimos la última observación para ver su formato
print(yi)#Imprimimos la última etiqueta para ver su formato